In [2]:
import json
import re

In [3]:
from selenium.common import exceptions

In [4]:
from linkedin_selenium import * #Don't do this *(Just for prototype)

In [5]:
user = os.environ.get('LINKEDIN_USER')
pwd = os.environ.get('LINKEDIN_PWD')
    
linkedin = LinkedIn(snooze=1)

23-Jul-19 22:50:55 [INFO] Setting Up LinkedIn


In [6]:
linkedin.sign_in(username=user,password=pwd)

23-Jul-19 22:50:56 [INFO] Login in LinkedIn
23-Jul-19 22:51:03 [INFO] Login Successful


In [7]:
ROOT_URL = 'https://www.linkedin.com'

In [8]:
df = pd.read_pickle('linkedin.pkl')

In [9]:
profile_index = np.random.randint(low=1, high=df.shape[0], size=1)[0]
profile = df.iloc[profile_index]

In [10]:
print(profile['profile_name'])
profile_url = f"{ROOT_URL}{profile['profile_url']}"

Jesper Horsted Sørensen


In [11]:
linkedin.driver.get(profile_url)

In [12]:
# has_about = True
# try:
#     linkedin.driver.find_element_by_class_name('lt-line-clamp__more').click()
# except (exceptions.ElementNotInteractableException,
#         exceptions.NoSuchElementException):
#     print('No Abouts')
#     has_about = False
    
# except exceptions.ElementClickInterceptedException:
#     linkedin.driver.find_element_by_class_name('msg-overlay-bubble-header').click()
#     linkedin.driver.find_element_by_class_name('lt-line-clamp__more').click()
    
# finally:
#     source = linkedin.driver.page_source

In [13]:
soup = BeautifulSoup(linkedin.driver.page_source,'lxml')

In [14]:
soup.find('a',{'data-control-name':'topcard_view_all_connections'})

<a class="ember-view" data-control-name="topcard_view_all_connections" href="/search/results/people/?facetConnectionOf=%5B%22ACoAAA0_VZ0BiK3-V_7KPtf8lcdqR4W4cjsL5Eo%22%5D&amp;facetNetwork=%5B%22F%22%2C%22S%22%5D&amp;origin=MEMBER_PROFILE_CANNED_SEARCH" id="ember96"> <span class="t-16 t-bold">
                    500+ connections
                  </span>
</a>

In [15]:
next_connections = soup.find('a',{'data-control-name':'topcard_view_all_connections'})

In [16]:
connections_count = next_connections.get_text(strip=True).split()[0]

In [17]:
next_connections['href']

'/search/results/people/?facetConnectionOf=%5B%22ACoAAA0_VZ0BiK3-V_7KPtf8lcdqR4W4cjsL5Eo%22%5D&facetNetwork=%5B%22F%22%2C%22S%22%5D&origin=MEMBER_PROFILE_CANNED_SEARCH'

In [18]:
next_url = f"{ROOT_URL}{next_connections['href']}"
linkedin.driver.get(next_url)

In [31]:
linkedin.driver.find_elements_by_class_name("search-s-facet__form")[0].click()

linkedin.driver.find_element_by_class_name("facet-collection-list__clear-button").click()

linkedin.driver.find_element_by_class_name("facet-collection-list__apply-button").click()

In [38]:
linkedin.driver.current_url

'https://www.linkedin.com/search/results/people/?facetConnectionOf=%5B%22ACoAAA0_VZ0BiK3-V_7KPtf8lcdqR4W4cjsL5Eo%22%5D&origin=FACETED_SEARCH&page=8'

In [39]:
linkedin.driver.get(linkedin.driver.current_url)

In [40]:
soup = BeautifulSoup(linkedin.driver.page_source,'lxml')
pattern = re.compile('^bpr-guid-*')
codes = soup.find_all('code',{'id':pattern})

data = [code.get_text(strip=True) for code in codes if 'metadata' in code.get_text(strip=True)]
# json.loads(data[-1])

In [41]:
d = json.loads(data[-1])

In [42]:
d

{'data': {'metadata': {'searchId': '81231e95-e896-428b-a690-2ac50d5a3f45',
   'totalResultCount': 535,
   'origin': 'FACETED_SEARCH',
   'numVisibleResults': 10,
   '$type': 'com.linkedin.voyager.search.BlendedSearchMetadata'},
  'entityUrn': 'urn:li:collectionResponse:z0xwllwvUe+YXWgr+xFqp73XqCTZssea8QLaDOr9g30=',
  'elements': [{'extendedElements': [],
    'elements': [{'memberDistance': {'value': 'DISTANCE_2',
       '$type': 'com.linkedin.voyager.common.MemberDistance'},
      'image': {'attributes': [{'sourceType': 'PROFILE_PICTURE',
         '*miniProfile': 'urn:li:fs_miniProfile:ACoAAAAKy8cB356FADCpGuSnX_LNkopyVjb5ra8',
         '$type': 'com.linkedin.voyager.common.ImageAttribute'}],
       'accessibilityTextAttributes': [],
       '$type': 'com.linkedin.voyager.common.ImageViewModel'},
      'targetUrn': 'urn:li:fs_miniProfile:ACoAAAAKy8cB356FADCpGuSnX_LNkopyVjb5ra8',
      'socialProofImagePile': [{'attributes': [{'sourceType': 'PROFILE_PICTURE',
          '*miniProfile': 'ur

In [44]:
d['data'].keys()

dict_keys(['metadata', 'entityUrn', 'elements', 'paging', '$type'])

In [22]:
d['data']['elements'][1].keys()

dict_keys(['extendedElements', 'elements', 'type', '$type'])

In [23]:
for data in d['data']['elements'][1]['elements']:
    print(data['memberDistance'])

{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}


In [24]:
for index, data in enumerate(d['data']['elements'][1]['elements'],1):
    print(f'\nConnection {index}')
    for key in data:
        print(f'key: {key}')
        print(data[key])
        print('--'*15)


Connection 1
key: memberDistance
{'value': 'DISTANCE_2', '$type': 'com.linkedin.voyager.common.MemberDistance'}
------------------------------
key: image
{'attributes': [{'sourceType': 'PROFILE_PICTURE', '*miniProfile': 'urn:li:fs_miniProfile:ACoAAAP5QFEBy8bLYYtn3qATtDNXjufoKP234Oo', '$type': 'com.linkedin.voyager.common.ImageAttribute'}], 'accessibilityTextAttributes': [], '$type': 'com.linkedin.voyager.common.ImageViewModel'}
------------------------------
key: targetUrn
urn:li:fs_miniProfile:ACoAAAP5QFEBy8bLYYtn3qATtDNXjufoKP234Oo
------------------------------
key: socialProofImagePile
[{'attributes': [{'sourceType': 'PROFILE_PICTURE', '*miniProfile': 'urn:li:fs_miniProfile:ACoAAAAkNzwBPMuArNwhiN6cbS6-nRK2_NcFQ30', '$type': 'com.linkedin.voyager.common.ImageAttribute'}], 'accessibilityTextAttributes': [], '$type': 'com.linkedin.voyager.common.ImageViewModel'}, {'attributes': [{'sourceType': 'PROFILE_PICTURE', '*miniProfile': 'urn:li:fs_miniProfile:ACoAAAB0cBMBt4y8SJDCXGtmdmad3gWuW

In [43]:
linkedin.sign_off(False)

23-Jul-19 23:04:31 [INFO] Logoff Successful


##### Notes still use pagnination to get everything

current url, check if it is close to the number of shown